# PRCP-1014

#### Vaccinatino Prediction Description- Project

Team Members: Vivek Viswanathan, Sushree Sarangi, Vartika Vishnoi, and Prabhati Acharya

In [1]:
#!pip install vecstack

In [2]:
# Import the necessary packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, RandomForestRegressor, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, StackingClassifier
import itertools
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.feature_selection import RFECV
from vecstack import stacking
from mlxtend.classifier import StackingCVClassifier
from mlxtend.plotting import plot_learning_curves, plot_decision_regions
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import NearestNeighbors
from sklearn.calibration import CalibratedClassifierCV
from pandas_profiling import ProfileReport
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn import tree
from sklearn.externals.six import StringIO
import graphviz

import warnings
warnings.simplefilter('ignore')

ModuleNotFoundError: No module named 'vecstack'

In [3]:
rand_state = 10
n_iterations = 5

In [4]:
# Load the data

Project_Data = pd.read_csv('features.csv')
#Project_Data_svm = pd.read_csv('bank-additional.csv', sep=';')

In [5]:
Project_Data.head(2)

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe


In [6]:
Project_Data.shape

(26707, 36)

In [7]:
level_data= pd.read_csv('labels.csv')

In [8]:
level_data.head(1)

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,0,0,0


In [9]:
level_data.shape

(26707, 3)

In [10]:
total_data=pd.concat([Project_Data, level_data],sort=False)

In [11]:
total_data.head(2)

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,NaN,NaN


In [12]:
total_data.shape

(53414, 38)

In [13]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53414 entries, 0 to 26706
Data columns (total 38 columns):
respondent_id                  53414 non-null int64
h1n1_concern                   26615 non-null float64
h1n1_knowledge                 26591 non-null float64
behavioral_antiviral_meds      26636 non-null float64
behavioral_avoidance           26499 non-null float64
behavioral_face_mask           26688 non-null float64
behavioral_wash_hands          26665 non-null float64
behavioral_large_gatherings    26620 non-null float64
behavioral_outside_home        26625 non-null float64
behavioral_touch_face          26579 non-null float64
doctor_recc_h1n1               24547 non-null float64
doctor_recc_seasonal           24547 non-null float64
chronic_med_condition          25736 non-null float64
child_under_6_months           25887 non-null float64
health_worker                  25903 non-null float64
health_insurance               14433 non-null float64
opinion_h1n1_vacc_effective  

In [14]:
total_data.isnull().sum()

respondent_id                      0
h1n1_concern                   26799
h1n1_knowledge                 26823
behavioral_antiviral_meds      26778
behavioral_avoidance           26915
behavioral_face_mask           26726
behavioral_wash_hands          26749
behavioral_large_gatherings    26794
behavioral_outside_home        26789
behavioral_touch_face          26835
doctor_recc_h1n1               28867
doctor_recc_seasonal           28867
chronic_med_condition          27678
child_under_6_months           27527
health_worker                  27511
health_insurance               38981
opinion_h1n1_vacc_effective    27098
opinion_h1n1_risk              27095
opinion_h1n1_sick_from_vacc    27102
opinion_seas_vacc_effective    27169
opinion_seas_risk              27221
opinion_seas_sick_from_vacc    27244
age_group                      26707
education                      28114
race                           26707
sex                            26707
income_poverty                 31130
m

In [15]:
total_data.nunique(dropna=False)

respondent_id                  26707
h1n1_concern                       5
h1n1_knowledge                     4
behavioral_antiviral_meds          3
behavioral_avoidance               3
behavioral_face_mask               3
behavioral_wash_hands              3
behavioral_large_gatherings        3
behavioral_outside_home            3
behavioral_touch_face              3
doctor_recc_h1n1                   3
doctor_recc_seasonal               3
chronic_med_condition              3
child_under_6_months               3
health_worker                      3
health_insurance                   3
opinion_h1n1_vacc_effective        6
opinion_h1n1_risk                  6
opinion_h1n1_sick_from_vacc        6
opinion_seas_vacc_effective        6
opinion_seas_risk                  6
opinion_seas_sick_from_vacc        6
age_group                          6
education                          5
race                               5
sex                                3
income_poverty                     4
m